# Extraction of missense variants per allele count octave

In [17]:
import hail as hl;
hl.init()

Running on Apache Spark version 3.3.4
SparkUI available at http://loic-powersfs-v2-motebook.us-central1-a.c.psychic-rhythm-198903.internal:4040
Welcome to
     __  __     <>__
    / /_/ /__  __/ /
   / __  / _ `/ / /
  /_/ /_/\_,_/_/_/   version 0.2.128-eead8100a1c1
LOGGING: writing to /home/jupyter/powerSFS/hail-20240401-0206-0.2.128-eead8100a1c1.log


In [19]:
# connect to gnomADv4.0 exomes
ht = hl.read_table('gs://gcp-public-data--gnomad/release/4.0/ht/exomes/gnomad.exomes.v4.0.sites.ht')

# setup list of variants' consequences
CSQ_LOF = [
    "transcript_ablation",
    "splice_acceptor_variant",
    "splice_donor_variant",
    "stop_gained",
    "frameshift_variant",
    "stop_lost"]

CSQ_MIS = ['missense_variant']
CSQ_SYN = ['synonymous_variant']
CSQ_ALL = CSQ_LOF + CSQ_MIS + CSQ_SYN

# functional variants
CSQ_FUNC = CSQ_LOF + CSQ_MIS

csqfunc = hl.literal(CSQ_FUNC)
selcsq = hl.literal(CSQ_ALL)

# keep variants that PASS and have AF>0 and most_severe_consequence is LoF or missense
ht = ht.filter((ht.freq[0].AF > 0) & (ht.filters.length()==0) & csqfunc.contains(ht.row.vep.most_severe_consequence))

# annotate variants with first gene matching most_severe_consequence
# first protein coding transcript matching worst consequence associated with a HGNC gene
# TODO should probably restrict to MANE transcript
# TODO should scan all transcripts in case of overlapping genes on different strands 
ht = ht.annotate(gene = ht.vep.transcript_consequences.find(lambda x: x.consequence_terms.contains(ht.vep.most_severe_consequence) & (x.biotype == 'protein_coding') & (x.gene_symbol_source == 'HGNC')).gene_symbol)

# only keep variants with a matching gene
ht = ht.filter(hl.is_defined(ht.gene))

# add allele count' octave
ht = ht.annotate(octave = hl.floor(hl.log(ht.freq[0].AC,2)))

# group by gene and octave and add some QC information
res = ht.group_by('gene','octave').aggregate(count = hl.agg.count(),
                                             AN_mean = hl.agg.mean(ht.freq[0].AN),
                                             AN_min = hl.agg.min(ht.freq[0].AN),
                                             AN_max = hl.agg.max(ht.freq[0].AN))

# export result
res.export('gs://vccrieg/powerSFSv2/gnomADv4_SFS_prototype.tsv')